In [3]:
import cython
print(cython.__version__)

0.28.2


In [4]:
import pandas as pd
import csv

train = pd.read_csv("../Part1-BOW/labeledTrainData.tsv", header=0, delimiter='\t', quoting=csv.QUOTE_NONE)
test = pd.read_csv("../Part1-BOW/testData.tsv", header=0, delimiter='\t', quoting=csv.QUOTE_NONE)
unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter='\t', quoting=csv.QUOTE_NONE)

print(train['review'].size, test['review'].size, unlabeled_train['review'].size)

25000 25000 50000


In [16]:
from bs4 import BeautifulSoup as bsoup
from nltk.corpus import stopwords
import re

def review_to_wordlist(review, remove_stopwords=False):
    review_text = bsoup(review, 'html.parser').get_text()
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [word for word in words if word not in stops]
        
    return words

In [17]:
import nltk

# for sentence tokenization
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [18]:
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    
    return sentences

In [19]:
# to see the progress of the loop
from tqdm import tqdm

sentences = []

for review in tqdm(train["review"]):
    sentences += review_to_sentences(review, tokenizer)
for review in tqdm(unlabeled_train["review"]):
    sentences += review_to_sentences(review, tokenizer)

  1%|▊                                                                                            | 205/25000 [00:00<00:52, 468.65it/s]C:\Users\nabhu\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
  6%|█████▋                                                                                      | 1544/25000 [00:03<00:51, 454.32it/s]C:\Users\nabhu\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\nabhu\Anaconda3\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that 

In [23]:
print(len(sentences))
print(sentences[10])

795538
['it', 'does', 'try', 'and', 'give', 'off', 'a', 'wholesome', 'message', 'and', 'ironically', 'mj', 's', 'bestest', 'buddy', 'in', 'this', 'movie', 'is', 'a', 'girl']


In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [24]:
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [25]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

model_name = "100features_40minwords_10context"
model.save(model_name)

C:\Users\nabhu\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-05-06 11:33:09,255 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-06 11:33:09,325 : INFO : collecting all words and their counts
2019-05-06 11:33:09,341 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-06 11:33:09,425 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-05-06 11:33:09,493 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-05-06 11:33:09,576 : INFO : PROGRESS: at sentence #30000, processed 671314 words, keeping 30034 word types
2019-05-06 11:33:09,645 : INFO : PROGRESS: at sentence #40000, processed 897814 words, keeping 34348 word types
2019-05-06 11:33:09,744 : INFO : PROGRESS: at sentence #50000, processe

2019-05-06 11:33:16,541 : INFO : PROGRESS: at sentence #680000, processed 15206483 words, keeping 115354 word types
2019-05-06 11:33:16,644 : INFO : PROGRESS: at sentence #690000, processed 15428676 words, keeping 116131 word types
2019-05-06 11:33:16,740 : INFO : PROGRESS: at sentence #700000, processed 15657382 words, keeping 116943 word types
2019-05-06 11:33:16,826 : INFO : PROGRESS: at sentence #710000, processed 15880371 words, keeping 117596 word types
2019-05-06 11:33:16,914 : INFO : PROGRESS: at sentence #720000, processed 16105658 words, keeping 118221 word types
2019-05-06 11:33:17,008 : INFO : PROGRESS: at sentence #730000, processed 16332039 words, keeping 118954 word types
2019-05-06 11:33:17,106 : INFO : PROGRESS: at sentence #740000, processed 16553072 words, keeping 119668 word types
2019-05-06 11:33:17,193 : INFO : PROGRESS: at sentence #750000, processed 16771399 words, keeping 120295 word types
2019-05-06 11:33:17,260 : INFO : PROGRESS: at sentence #760000, processe

2019-05-06 11:34:02,574 : INFO : EPOCH 3 - PROGRESS: at 23.98% examples, 500142 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:03,585 : INFO : EPOCH 3 - PROGRESS: at 28.70% examples, 513497 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:04,587 : INFO : EPOCH 3 - PROGRESS: at 33.72% examples, 527574 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:05,594 : INFO : EPOCH 3 - PROGRESS: at 38.51% examples, 536733 words/s, in_qsize 8, out_qsize 0
2019-05-06 11:34:06,605 : INFO : EPOCH 3 - PROGRESS: at 41.76% examples, 524257 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:07,603 : INFO : EPOCH 3 - PROGRESS: at 44.74% examples, 511172 words/s, in_qsize 6, out_qsize 1
2019-05-06 11:34:08,606 : INFO : EPOCH 3 - PROGRESS: at 47.95% examples, 503080 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:09,609 : INFO : EPOCH 3 - PROGRESS: at 53.43% examples, 518117 words/s, in_qsize 7, out_qsize 0
2019-05-06 11:34:10,639 : INFO : EPOCH 3 - PROGRESS: at 57.85% examples, 520742 words/s, in_qsiz

2019-05-06 11:34:59,259 : INFO : saved 100features_40minwords_10context


In [27]:
model.wv.doesnt_match("man woman child kitchen".split())

'kitchen'

In [29]:
model.wv.most_similar('india')

[('europe', 0.8573665618896484),
 ('pakistan', 0.8388616442680359),
 ('spain', 0.8369898200035095),
 ('france', 0.8231020569801331),
 ('china', 0.8115490674972534),
 ('greece', 0.8082659244537354),
 ('england', 0.8039284944534302),
 ('germany', 0.8038352131843567),
 ('italy', 0.7948799729347229),
 ('poland', 0.7881669998168945)]

In [34]:
model.wv.most_similar(positive=['woman', 'prince'], negative=['man'])[0]

('princess', 0.7912745475769043)